# Radical Pilot Tutorial


## 1. Installation (Not needed on Chameleon)

Not required on <http://sc15.radical-cybertools.org/>

If you want to install Radical-Pilot locally:

    pip install radical.pilot
    pip install radical.utils
    
## 2. Radical Pilot Setup

Import dependency and create callbacks for state update

Documentation: http://radicalpilot.readthedocs.org/en/latest/machconf.html#preconfigured-resources

Mongo:
    
           monogodb
           $ db.createCollection("sc15-tutorial", {capped: true, size: 100000, max: 100});

In [4]:
import os, sys
import commands
import radical.pilot as rp

#os.environ["RADICAL_PILOT_DBURL"]="mongodb://localhost/radicalpilot"

os.environ["RADICAL_PILOT_DBURL"]="mongodb://ec2-54-221-194-147.compute-1.amazonaws.com:24242/giannis"
os.environ["RADICAL_PILOT_VERBOSE"]="DEBUG"

""" DESCRIPTION: Tutorial 1: A Simple Workload consisting of a Bag-of-Tasks
"""

def pilot_state_cb (pilot, state):
    if not pilot:
        return
    print "[Callback]: ComputePilot '%s' state: %s." % (pilot.uid, state)
    if state == rp.FAILED:
        sys.exit (1)

def unit_state_cb (unit, state):
    if not unit:
        return

    global CNT
    print "[Callback]: unit %s on %s: %s." % (unit.uid, unit.pilot_id, state)
    if state == rp.FAILED:
        print "stderr: %s" % unit.stderr
        sys.exit(2)

In [5]:
commands.getstatusoutput('echo $RADICAL_PILOT_VERBOSE')

(0, 'DEBUG')

## 3. Submit Pilot

### 3.1 Create a new Session and Pilot-Manager. 

In [14]:
session = rp.Session()
c = rp.Context('ssh')
c.user_id = "radical"
session.add_context(c)
pmgr = rp.PilotManager(session=session)

print "Session id: %s Pilot Manager: %s" % (session.uid, str(pmgr.as_dict()))

Session id: rp.session.radical-1.radical.016739.0006 Pilot Manager: {'uid': 'pmgr.0002'}


### 3.2 Submit Pilot and initialize Unit Manager

In [15]:
pdesc = rp.ComputePilotDescription ()
pdesc.resource = "local.localhost"  # NOTE: This is a "label", not a hostname
pdesc.runtime  = 10 # minutes
pdesc.cores    = 16
pdesc.cleanup  = False
# submit the pilot.
pilot = pmgr.submit_pilots(pdesc)

# Combine the ComputePilot, the ComputeUnits and a scheduler via
# a UnitManager object.
print "Initializing Unit Manager ..."
umgr = rp.UnitManager (session=session,
                       scheduler=rp.SCHED_DIRECT_SUBMISSION)

print "Registering Compute Pilot with Unit Manager ..."
umgr.add_pilots(pilot)

Initializing Unit Manager ...
Registering Compute Pilot with Unit Manager ...


### 3.3 Submit Compute Units

In [16]:
NUMBER_JOBS  = 2 # the total number of cus to run
cudesc_list = []
for i in range(NUMBER_JOBS):
    # -------- BEGIN USER DEFINED CU DESCRIPTION --------- #
    cudesc = rp.ComputeUnitDescription()
    cudesc.environment = {'CU_NO': i}
    cudesc.executable  = "/bin/echo"
    cudesc.arguments   = ['I am CU number $CU_NO']
    cudesc.cores       = 1
    # -------- END USER DEFINED CU DESCRIPTION --------- #
    cudesc_list.append(cudesc)
# Submit the previously created ComputeUnit descriptions to the
# PilotManager. This will trigger the selected scheduler to start
# assigning ComputeUnits to the ComputePilots.
print "Submit Compute Units to Unit Manager ..."
cu_set = umgr.submit_units (cudesc_list)
print "Waiting for CUs to complete ..."
umgr.wait_units()
print "All CUs completed successfully!"
for cu in cu_set:
    print cu.stdout

Submit Compute Units to Unit Manager ...
Waiting for CUs to complete ...
All CUs completed successfully!
I am CU number 0

I am CU number 1



### 3.4 Close Session

In [12]:
session.close()